In [37]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString,MultiLineString


In [8]:
estaciones = pd.read_csv('../data/estaciones_ecobici_api.csv')
estaciones = estaciones.reindex(columns = ['station_id','lat','lon'])
estaciones

,station_id,lat,lon
0,2,-34.592423,-58.374715
1,3,-34.611032,-58.368260
2,4,-34.601726,-58.368763
3,5,-34.580550,-58.420954
4,6,-34.628526,-58.369758
...,...,...,...
393,444,-34.608678,-58.371008
394,448,-34.582618,-58.378731
395,449,-34.628937,-58.463661
396,450,-23.556362,-46.662562


In [11]:
viajes_posibles = pd.DataFrame()
for origen in estaciones.index:
    if origen in [100,200,300]:
        print(origen)
    for destino in estaciones.index:
        if origen != destino:
    
            viaje_id = '%i_%i' % (origen,destino) 
            df_o = estaciones.loc[[origen],:]
            df_o.columns = [col+'_o' for col in df_o.columns]
            df_o.index = [viaje_id]

            df_d = estaciones.loc[[destino],:]
            df_d.columns = [col+'_d' for col in df_d.columns]
            df_d.index = [viaje_id]
            
            df_viaje = pd.concat([df_o,df_d],axis=1)
            
            viajes_posibles = viajes_posibles.append(df_viaje)

100
200
300


In [13]:
viajes_posibles.head()

,station_id_o,lat_o,lon_o,station_id_d,lat_d,lon_d
0_1,2,-34.592423,-58.374715,3,-34.611032,-58.368260
0_2,2,-34.592423,-58.374715,4,-34.601726,-58.368763
0_3,2,-34.592423,-58.374715,5,-34.580550,-58.420954
0_4,2,-34.592423,-58.374715,6,-34.628526,-58.369758
0_5,2,-34.592423,-58.374715,7,-34.606498,-58.381098


In [14]:
viajes_posibles = viajes_posibles.reset_index()
viajes_posibles.head()

,index,station_id_o,lat_o,lon_o,station_id_d,lat_d,lon_d
0,0_1,2,-34.592423,-58.374715,3,-34.611032,-58.368260
1,0_2,2,-34.592423,-58.374715,4,-34.601726,-58.368763
2,0_3,2,-34.592423,-58.374715,5,-34.580550,-58.420954
3,0_4,2,-34.592423,-58.374715,6,-34.628526,-58.369758
4,0_5,2,-34.592423,-58.374715,7,-34.606498,-58.381098


In [15]:
viajes_posibles = viajes_posibles.rename(columns = {'index':'viaje_id'})

In [23]:
viajes_realizados = gpd.read_file('../carto/ecobici/ecobici.shp')
viajes_realizados = viajes_realizados.reindex(columns = ['id_estacio','id_estac_1','n'])
viajes_realizados.columns = ['station_id_o','station_id_d','n']
viajes_realizados.head()

,station_id_o,station_id_d,n
0,2.0,2.0,141
1,2.0,4.0,68
2,2.0,5.0,28
3,2.0,6.0,20
4,2.0,7.0,3


In [25]:
viajes_posibles = viajes_posibles.merge(viajes_realizados,on = ['station_id_o','station_id_d'])

In [27]:
viajes_posibles.to_csv('../data/viajes_posibles_ecobici.csv',index=False)

In [35]:
import osmnx as ox
import networkx as nx


In [32]:
G = ox.load_graphml(filename='caba_callejero.graphml',folder='../carto')
edges = ox.graph_to_gdfs(G, nodes=False).set_index(['u', 'v']).sort_index()


In [33]:
def convertir_OD_en_ruta(o_latlong,d_latlong,tipo,G=G,edges=edges,plot=False):
    # busco el nodo mas cercano de ambos puntos 
    orig_node = ox.get_nearest_node(G, o_latlong)
    dest_node = ox.get_nearest_node(G, d_latlong)
    
    #obtengo la ruta
    route = nx.shortest_path(G, orig_node, 
                         dest_node)
    
    #nodos en pares
    route_pairwise = zip(route[:-1], route[1:])
    if plot:
        fig, ax = ox.plot_graph_route(G, route, node_size=0)
    if tipo == 'arcos':
        tramos_unicos = [edges.loc[uv, 'osmid'].iloc[0] for uv in route_pairwise]
        tramos_unicos = pd.Series(tramos_unicos).map(str).unique()
        
        return tramos_unicos
    elif tipo == 'linea':
        lines = [edges.loc[uv, 'geometry'].iloc[0] for uv in route_pairwise]
        
        return MultiLineString(lines)
    else:
        return None
    
def aplicar_convertir_OD_en_ruta_linea(fila):
    return convertir_OD_en_ruta(o_latlong = (fila.lat_o,fila.lon_o),
                                d_latlong = (fila.lat_d,fila.lon_d),
                                tipo='linea',
                                G=G,
                                edges=edges)

def aplicar_convertir_OD_en_ruta_arcos(fila):
    return convertir_OD_en_ruta(o_latlong = (fila.lat_o,fila.lon_o),
                                d_latlong = (fila.lat_d,fila.lon_d),
                                tipo='arcos',
                                G=G,
                                edges=edges)

In [38]:
viajes_posibles['geometry'] = viajes_posibles.apply(aplicar_convertir_OD_en_ruta_linea,axis=1)


In [41]:
viajes_posibles_tramos = viajes_posibles.copy()

In [45]:
lineas_sin_ponderar = gpd.GeoDataFrame(viajes_posibles,geometry = 'geometry',crs={'init' :'epsg:4326'})
lineas_sin_ponderar.to_file('../carto/viajes_ecobici_linea_sin_ponderar')

In [47]:
import numpy as np

In [48]:
lineas_ponderadas = viajes_posibles.reindex(columns = ['viaje_id','n','geometry'])
lineas_ponderadas = lineas_ponderadas.loc[np.repeat(lineas_ponderadas.index,lineas_ponderadas.n),:]
lineas_ponderadas = gpd.GeoDataFrame(lineas_ponderadas,geometry = 'geometry',crs={'init' :'epsg:4326'})
lineas_ponderadas

,viaje_id,n,geometry
0,0_2,68,"MULTILINESTRING ((-58.37491 -34.59297, -58.374..."
0,0_2,68,"MULTILINESTRING ((-58.37491 -34.59297, -58.374..."
0,0_2,68,"MULTILINESTRING ((-58.37491 -34.59297, -58.374..."
0,0_2,68,"MULTILINESTRING ((-58.37491 -34.59297, -58.374..."
0,0_2,68,"MULTILINESTRING ((-58.37491 -34.59297, -58.374..."
...,...,...,...
15173,393_195,53,"MULTILINESTRING ((-58.37095 -34.60887, -58.370..."
15173,393_195,53,"MULTILINESTRING ((-58.37095 -34.60887, -58.370..."
15173,393_195,53,"MULTILINESTRING ((-58.37095 -34.60887, -58.370..."
15173,393_195,53,"MULTILINESTRING ((-58.37095 -34.60887, -58.370..."


In [49]:
lineas_ponderadas.to_file('../carto/viajes_ecobici_linea_ponderadas')

In [50]:
viajes_posibles_tramos = viajes_posibles_tramos.drop('geometry',axis=1)
viajes_posibles_tramos['arcos'] = viajes_posibles_tramos.apply(aplicar_convertir_OD_en_ruta_arcos,axis=1)


In [52]:
viajes_posibles_tramos.to_csv('viajes_posibles_tramos.csv',index=False)

In [54]:
viajes_posibles_tramos.head()

,viaje_id,station_id_o,lat_o,lon_o,station_id_d,lat_d,lon_d,n,arcos
0,0_2,2,-34.592423,-58.374715,4,-34.601726,-58.368763,68,"[695766973, 457551190, 223186335, 79876738, 46..."
1,0_3,2,-34.592423,-58.374715,5,-34.580550,-58.420954,28,"[695766973, 457551190, 223186335, 79876738, 69..."
2,0_4,2,-34.592423,-58.374715,6,-34.628526,-58.369758,20,"[695766973, 457551190, 223186335, 79876738, 46..."
3,0_5,2,-34.592423,-58.374715,7,-34.606498,-58.381098,3,"[450495577, 450495575, 289835399, 603920464, 1..."
4,0_6,2,-34.592423,-58.374715,8,-34.609422,-58.389336,19,"[450495577, 450495575, 289835399, 603920464, 1..."


In [56]:
tramos_ponderados = pd.DataFrame()
for i in viajes_posibles_tramos.index:
    df = pd.DataFrame({'osmid':viajes_posibles_tramos.arcos.loc[i],
                       'viaje_id':viajes_posibles_tramos.viaje_id.loc[i]})
    tramos_ponderados = tramos_ponderados.append(df)
    

In [58]:
tramos_ponderados.to_csv('tramos_ponderados.csv',index=False)

In [79]:
edges_to_merge = edges.reindex(columns = ['osmid','geometry']).copy()
#merge(tramos_ponderados)

In [78]:
tramos_ponderados = tramos_ponderados.groupby('osmid').count().reset_index()
tramos_ponderados.columns = ['osmid','n']

In [80]:
edges_to_merge.osmid = edges_to_merge.osmid.map(str)

In [81]:
edges_to_merge = edges_to_merge.merge(tramos_ponderados,how='right')

In [82]:
edges_to_merge.to_file('../carto/viajes_ecobici_tramos')

In [83]:
edges_to_merge

,osmid,geometry,n
0,313024196,"LINESTRING (-58.38205 -34.59159, -58.38185 -34...",62
1,451645199,"LINESTRING (-58.38205 -34.59159, -58.38206 -34...",140
2,275088523,"LINESTRING (-58.38172 -34.60613, -58.38172 -34...",64
3,275088563,"LINESTRING (-58.38172 -34.60613, -58.38208 -34...",46
4,96867643,"LINESTRING (-58.38098 -34.62216, -58.38119 -34...",133
...,...,...,...
6585,695765243,"LINESTRING (-58.37210 -34.59165, -58.37215 -34...",101
6586,695765245,"LINESTRING (-58.37210 -34.59165, -58.37226 -34...",54
6587,18827289,"LINESTRING (-58.37197 -34.59235, -58.37190 -34...",88
6588,419996419,"LINESTRING (-58.37215 -34.58713, -58.37209 -34...",17


In [ ]:
#traer las geom de los osmid de edges